<a href="https://colab.research.google.com/github/steinruck/WANG-690-FALL-2020/blob/master/Assignment_06/Assignment_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 06

1. Use Pandas read_html() function to retrive the HTML tables as dataframes from the web page: https://www.genealogybranches.com/censuscosts.html

In [1]:
import pandas as pd
import html5lib
url = 'https://www.genealogybranches.com/censuscosts.html'
dfs = pd.read_html(url)
type(dfs)

list

In [3]:
dfs.head()

AttributeError: ignored

2. Find out how many HTML tables Pandas retrieves from the web page. Find out which dataframe contain the data and use it for further processing.

In [4]:
len(dfs)

2

In [5]:
dfs = pd.read_html(url)[0]
dfs.head()

,0,1,2,3
0,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents


In [6]:
len(dfs)

24

3. Display and explore the data (rows, columns, etc.) and determine the data quality (bad rows, bad columns, bad elements, null values, etc.)
4. Document the data quality issues using a Markdown cell so the reader understands what the problems are.

In [7]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       24 non-null     object
 1   1       24 non-null     object
 2   2       24 non-null     object
 3   3       24 non-null     object
dtypes: object(4)
memory usage: 896.0+ bytes


In [8]:
  dfs.columns

Int64Index([0, 1, 2, 3], dtype='int64')

Not registering the column titles as column titles. instead using int

In [9]:
dfs.shape

(24, 4)

In [10]:
type(dfs)

pandas.core.frame.DataFrame

In [11]:
dfs.head(1)

,0,1,2,3
0,Census Year,Total Population,Census Cost,Average Cost Per Person


In [12]:
header = dfs.iloc[0]
header

0                Census Year
1           Total Population
2                Census Cost
3    Average Cost Per Person
Name: 0, dtype: object

In [13]:
dfs = dfs.iloc[1:]
dfs.head()

,0,1,2,3
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents


In [14]:
type(header)

pandas.core.series.Series

In [15]:
header_list = list(header)
header_list

['Census Year', 'Total Population', 'Census Cost', 'Average Cost Per Person']

In [16]:
dfs.columns = header_list
dfs.head()

,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents


In [17]:
dfs.describe

<bound method NDFrame.describe of    Census Year Total Population     Census Cost Average Cost Per Person
1         1790          3929214         $44,377              1.13 cents
2         1800          5308483         $66,109              1.24 cents
3         1810          7239881        $178,445              2.46 cents
4         1820          9633822        $208,526              2.16 cents
5         1830         12866020        $378,545              2.94 cents
6         1840         17069458        $833,371              4.88 cents
7         1850         23191876      $1,423,351              6.14 cents
8         1860         31443321      $1,969,377              6.26 cents
9         1870         38558371      $3,421,198              8.87 cents
10        1880         50155783      $5,790,678             11.54 cents
11        1890         62979766     $11,547,127             18.33 cents
12        1900         76303387     $11,854,000             15.54 cents
13        1910         9197226

Doesn't display calculations because there is no uniformity to the data. Need to remove $ and commas. Convert cents to dollars. Remove extra characters.

In [18]:
dfs.describe()

,Census Year,Total Population,Census Cost,Average Cost Per Person
count,23,23,23,23
unique,23,23,23,23
top,1990,179323175,"$127,934,000",$4.76
freq,1,1,1,1


In [19]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 1 to 23
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Census Year              23 non-null     object
 1   Total Population         23 non-null     object
 2   Census Cost              23 non-null     object
 3   Average Cost Per Person  23 non-null     object
dtypes: object(4)
memory usage: 868.0+ bytes


In [20]:
dfs.columns

Index(['Census Year', 'Total Population', 'Census Cost',
       'Average Cost Per Person'],
      dtype='object')

5. Write a function named cleanse_year() that takes a string as input and removes any asterisks in the string and return the cleansed string. Test the function by using test strings (for example, "1989*", "20*10")

In [82]:
df = pd.DataFrame({'Year' : ['1989*', '20*10'], 'Money': ['$10,000', '$14 billion'], 'Math': ['$1.84', '1.12 cents']})
df

,Year,Money,Math
0,1989*,"$10,000",$1.84
1,20*10,$14 billion,1.12 cents


In [69]:
def clean_year(year_str):
    return str(year_str.replace('*', ''))
df['Year'].apply(clean_year)
    #df['Year'] = df['Year'].str.replace('*', '')
    #result
#df

0    1989
1    2010
Name: Year, dtype: object

In [48]:
#df['Money'] = df['Money'].str.replace('$', '')
#df['Money'] = df['Money'].str.replace(',', '')
#df['Money'] = df['Money'].str.replace(' billion', '000000000')
#df

,Year,Money,Math
0,1989,10000,$1.84
1,2010,14000000000,1.12 cents


In [83]:
def clean_numbers(row):
    return float(row.replace('$', '').replace(',', '').replace(' billion', '000000000'))
df['Money'].apply(clean_numbers)
#df['Math'] = df['Math'].str.replace('$', '')
#df


0    1.000000e+04
1    1.400000e+10
Name: Money, dtype: float64

In [55]:
df['Math'] = df['Math'].str.replace('cents', '')
df

,Year,Money,Math
0,1989*,"$10,000",1.84
1,20*10,$14 billion,1.12


In [56]:
df['Math'] = df['Math'].astype(float)
df

,Year,Money,Math
0,1989*,"$10,000",1.84
1,20*10,$14 billion,1.12


In [21]:
dfs['Census Year']

1      1790
2      1800
3      1810
4      1820
5      1830
6      1840
7      1850
8      1860
9      1870
10     1880
11     1890
12     1900
13     1910
14     1920
15     1930
16     1940
17     1950
18     1960
19     1970
20     1980
21     1990
22     2000
23    2010*
Name: Census Year, dtype: object

In [22]:
dfs.index

RangeIndex(start=1, stop=24, step=1)

In [23]:
dfs.columns[0]

'Census Year'

In [64]:
def cleanse_year():
    dfs['Census Year'] = dfs['Census Year'].str.replace('*', '')
    dfs['Census Year']
dfs.apply(cleanse_year)

TypeError: ignored

6. Cleanse the column "Census Year" using the function cleanse_year() and Pandas's apply() function.


7. Repeat this process for column "Total Population", "Census Cost", "Average Cost Per Person". (define, test, and apply the function)

In [28]:
dfs['Total Population']


1       3929214
2       5308483
3       7239881
4       9633822
5      12866020
6      17069458
7      23191876
8      31443321
9      38558371
10     50155783
11     62979766
12     76303387
13     91972266
14    105710620
15    122775046
16    131669275
17    151325798
18    179323175
19    203302031
20    226542199
21    248718301
22    281421906
23    308745538
Name: Total Population, dtype: object

In [32]:
dfs['Census Cost'].replace('$', '', inplace=True)
dfs


,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents
6,1840,17069458,"$833,371",4.88 cents
7,1850,23191876,"$1,423,351",6.14 cents
8,1860,31443321,"$1,969,377",6.26 cents
9,1870,38558371,"$3,421,198",8.87 cents
10,1880,50155783,"$5,790,678",11.54 cents


In [ ]:
dfs['Average Cost Per Person']


df["ERR"] = df["ERR"].astype(float)


8. After all columns are cleansed, save the clenased dataframe to a file named "census_cost_cleansed.csv" using CSV format.
9. Use Pandas to read the saved cleansed file and explore to make sure it is clean.
10. Updload both the Jupyter notebook and the cleanse data file to GitHub and submit the GitHub link in BlackBoard.